In [3]:
# Sales Forecasting with MLOps Pipeline - Complete Implementation for Colab
# Demonstrates time series analysis, forecasting, and production deployment

# Install required packages
!pip install statsmodels plotly xgboost lightgbm shap

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Time series and forecasting
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

class SalesForecastingSystem:
    def __init__(self):
        self.models = {}
        self.best_model = None
        self.feature_columns = []
        self.target_column = 'sales'
        self.date_column = 'date'
        self.scaler = None
        self.model_metadata = {}

    def generate_realistic_sales_data(self, start_date='2020-01-01', periods=1095):
        """Generate realistic sales data with trend, seasonality, and external factors"""
        print(f"Generating {periods} days of realistic sales data...")
        np.random.seed(42)

        # Create date range
        dates = pd.date_range(start=start_date, periods=periods, freq='D')

        # Base trend (growing business)
        trend = np.linspace(1000, 2000, periods)

        # Seasonal patterns
        # Weekly seasonality (higher sales on weekends)
        weekly_season = 300 * np.sin(2 * np.pi * np.arange(periods) / 7 + np.pi/2)

        # Monthly seasonality (higher sales at month end)
        monthly_season = 150 * np.sin(2 * np.pi * np.arange(periods) / 30)

        # Yearly seasonality (holiday seasons)
        yearly_season = 400 * np.sin(2 * np.pi * np.arange(periods) / 365.25 - np.pi/2)

        # Random noise
        noise = np.random.normal(0, 100, periods)

        # Special events (Black Friday, Christmas, etc.)
        special_events = np.zeros(periods)
        for year in range(2020, 2024):
            # Black Friday (late November)
            black_friday = pd.Timestamp(f'{year}-11-25')
            if black_friday in dates:
                idx = dates.get_loc(black_friday)
                special_events[idx:min(idx+4, periods)] = 800

            # Christmas season (December)
            christmas_start = pd.Timestamp(f'{year}-12-01')
            christmas_end = pd.Timestamp(f'{year}-12-25')
            if christmas_start in dates:
                start_idx = dates.get_loc(christmas_start)
                end_idx = min(dates.get_loc(christmas_end) if christmas_end in dates else periods, periods)
                special_events[start_idx:end_idx] = 600

        # Combine all components
        sales = trend + weekly_season + monthly_season + yearly_season + noise + special_events
        sales = np.maximum(sales, 100)  # Ensure positive sales

        # Create DataFrame with features
        df = pd.DataFrame({
            'date': dates,
            'sales': sales,
            'day_of_week': dates.dayofweek,
            'day_of_month': dates.day,
            'month': dates.month,
            'quarter': dates.quarter,
            'year': dates.year,
            'is_weekend': (dates.dayofweek >= 5).astype(int),
            'is_month_end': (dates.day >= 28).astype(int),
            'is_holiday_season': ((dates.month == 11) | (dates.month == 12)).astype(int),
            'week_of_year': dates.isocalendar().week
        })

        # Add external factors
        df['temperature'] = 20 + 15 * np.sin(2 * np.pi * np.arange(periods) / 365.25) + np.random.normal(0, 5, periods)
        df['marketing_spend'] = np.random.gamma(2, 100, periods) + 200 * df['is_holiday_season']
        df['competitor_price'] = 100 + np.random.normal(0, 10, periods)
        df['economic_index'] = 100 + np.cumsum(np.random.normal(0, 0.1, periods))

        # Add lagged features
        for lag in [1, 7, 30]:
            df[f'sales_lag_{lag}'] = df['sales'].shift(lag)

        # Add rolling statistics
        for window in [7, 30]:
            df[f'sales_ma_{window}'] = df['sales'].rolling(window).mean()
            df[f'sales_std_{window}'] = df['sales'].rolling(window).std()

        print(f"Generated dataset shape: {df.shape}")
        print(f"Date range: {df['date'].min()} to {df['date'].max()}")
        print(f"Average daily sales: ${df['sales'].mean():.2f}")

        return df

    def perform_comprehensive_eda(self, df):
        """Comprehensive EDA for time series sales data"""
        print("\n" + "="*60)
        print("SALES FORECASTING - EXPLORATORY DATA ANALYSIS")
        print("="*60)

        # Basic statistics
        print(f"Dataset shape: {df.shape}")
        print(f"Date range: {df['date'].min()} to {df['date'].max()}")
        print(f"Total sales: ${df['sales'].sum():,.2f}")
        print(f"Average daily sales: ${df['sales'].mean():.2f}")
        print(f"Sales standard deviation: ${df['sales'].std():.2f}")

        # Create comprehensive visualizations
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=['Sales Over Time', 'Sales Distribution',
                          'Seasonal Decomposition', 'Day of Week Pattern',
                          'Monthly Sales Pattern', 'Sales vs External Factors'],
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"colspan": 2}, None],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )

        # 1. Sales over time
        fig.add_trace(
            go.Scatter(x=df['date'], y=df['sales'], mode='lines', name='Daily Sales',
                      line=dict(color='blue')),
            row=1, col=1
        )

        # 2. Sales distribution
        fig.add_trace(
            go.Histogram(x=df['sales'], name='Sales Distribution', nbinsx=50),
            row=1, col=2
        )

        # 3. Seasonal decomposition (trend component)
        try:
            decomposition = seasonal_decompose(df.set_index('date')['sales'], model='additive', period=365)
            fig.add_trace(
                go.Scatter(x=df['date'], y=decomposition.trend, mode='lines',
                          name='Trend', line=dict(color='red')),
                row=2, col=1
            )
        except:
            print("Note: Seasonal decomposition requires at least 2 full periods")

        # 4. Day of week pattern
        dow_pattern = df.groupby('day_of_week')['sales'].mean()
        dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        fig.add_trace(
            go.Bar(x=dow_labels, y=dow_pattern.values, name='Avg Sales by Day',
                   marker_color='green'),
            row=3, col=1
        )

        # 5. Monthly pattern
        monthly_pattern = df.groupby('month')['sales'].mean()
        month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        fig.add_trace(
            go.Bar(x=month_labels, y=monthly_pattern.values, name='Avg Sales by Month',
                   marker_color='orange'),
            row=3, col=2
        )

        fig.update_layout(height=1000, showlegend=False, title_text="Sales Data Analysis Dashboard")
        fig.show()

        # Statistical insights
        print("\nKey Insights:")
        print(f"1. Highest sales day of week: {dow_labels[dow_pattern.idxmax()]} (${dow_pattern.max():.2f})")
        print(f"2. Lowest sales day of week: {dow_labels[dow_pattern.idxmin()]} (${dow_pattern.min():.2f})")
        print(f"3. Best month: {month_labels[monthly_pattern.idxmax()-1]} (${monthly_pattern.max():.2f})")
        print(f"4. Weekend vs weekday sales: ${df[df['is_weekend']==1]['sales'].mean():.2f} vs ${df[df['is_weekend']==0]['sales'].mean():.2f}")
        print(f"5. Holiday season vs regular: ${df[df['is_holiday_season']==1]['sales'].mean():.2f} vs ${df[df['is_holiday_season']==0]['sales'].mean():.2f}")

        # Stationarity test
        result = adfuller(df['sales'].dropna())
        print(f"\nStationarity Test (Augmented Dickey-Fuller):")
        print(f"ADF Statistic: {result[0]:.4f}")
        print(f"p-value: {result[1]:.4f}")
        print("Series is stationary" if result[1] <= 0.05 else "Series is non-stationary")

        return df

    def advanced_feature_engineering(self, df):
        """Create advanced features for time series forecasting"""
        print("\nPerforming advanced feature engineering...")

        df = df.copy()

        # Cyclical encoding for better ML performance
        df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['day_of_month_sin'] = np.sin(2 * np.pi * df['day_of_month'] / 31)
        df['day_of_month_cos'] = np.cos(2 * np.pi * df['day_of_month'] / 31)

        # Growth and trend features
        df['sales_growth_1d'] = df['sales'].pct_change(periods=1)
        df['sales_growth_7d'] = df['sales'].pct_change(periods=7)
        df['sales_growth_30d'] = df['sales'].pct_change(periods=30)

        # Interaction features
        df['marketing_temp_interaction'] = df['marketing_spend'] * df['temperature']
        df['weekend_holiday_interaction'] = df['is_weekend'] * df['is_holiday_season']
        df['price_marketing_ratio'] = df['competitor_price'] / (df['marketing_spend'] + 1)

        # Advanced rolling statistics
        for window in [14, 60, 90]:
            df[f'sales_ma_{window}'] = df['sales'].rolling(window).mean()
            df[f'sales_std_{window}'] = df['sales'].rolling(window).std()
            df[f'sales_min_{window}'] = df['sales'].rolling(window).min()
            df[f'sales_max_{window}'] = df['sales'].rolling(window).max()

        # Exponential smoothing features
        df['sales_ema_7'] = df['sales'].ewm(span=7).mean()
        df['sales_ema_30'] = df['sales'].ewm(span=30).mean()

        print(f"Feature engineering complete. Dataset shape: {df.shape}")

        return df

    def prepare_ml_datasets(self, df, forecast_horizon=30, test_split=0.2):
        """Prepare datasets for machine learning models"""
        print(f"\nPreparing ML datasets with {forecast_horizon} day forecast horizon...")

        # Define feature columns (exclude target and date)
        exclude_cols = ['date', 'sales']
        feature_cols = [col for col in df.columns if col not in exclude_cols]

        # Remove rows with NaN values (due to lagged features)
        df_clean = df.dropna().reset_index(drop=True)

        print(f"Cleaned dataset shape: {df_clean.shape}")
        print(f"Features used: {len(feature_cols)}")

        # Time-based split (important for time series)
        split_idx = int(len(df_clean) * (1 - test_split))

        train_df = df_clean.iloc[:split_idx].copy()
        test_df = df_clean.iloc[split_idx:].copy()

        X_train = train_df[feature_cols]
        y_train = train_df['sales']
        X_test = test_df[feature_cols]
        y_test = test_df['sales']

        self.feature_columns = feature_cols

        print(f"Training set: {X_train.shape[0]} samples")
        print(f"Test set: {X_test.shape[0]} samples")
        print(f"Test period: {test_df['date'].min()} to {test_df['date'].max()}")

        return X_train, X_test, y_train, y_test, train_df, test_df

    def train_multiple_models(self, X_train, y_train, X_test, y_test):
        """Train multiple forecasting models and compare performance"""
        print("\n" + "="*50)
        print("TRAINING MULTIPLE FORECASTING MODELS")
        print("="*50)

        # Define models
        models = {
            'Linear Regression': LinearRegression(),
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
            'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, verbose=0),
            'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
        }

        results = {}

        for name, model in models.items():
            print(f"\nTraining {name}...")

            # Train model
            start_time = datetime.now()
            model.fit(X_train, y_train)
            training_time = (datetime.now() - start_time).total_seconds()

            # Make predictions
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)

            # Calculate metrics
            train_mae = mean_absolute_error(y_train, y_pred_train)
            test_mae = mean_absolute_error(y_test, y_pred_test)
            train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
            test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
            test_r2 = r2_score(y_test, y_pred_test)

            # Calculate MAPE (Mean Absolute Percentage Error)
            test_mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

            results[name] = {
                'model': model,
                'train_mae': train_mae,
                'test_mae': test_mae,
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'test_r2': test_r2,
                'test_mape': test_mape,
                'training_time': training_time,
                'predictions': y_pred_test
            }

            print(f"Test MAE: ${test_mae:.2f}")
            print(f"Test RMSE: ${test_rmse:.2f}")
            print(f"Test R²: {test_r2:.4f}")
            print(f"Test MAPE: {test_mape:.2f}%")
            print(f"Training time: {training_time:.2f} seconds")

        # Select best model based on test MAE
        best_model_name = min(results.keys(), key=lambda x: results[x]['test_mae'])
        self.best_model = results[best_model_name]['model']

        print(f"\n🏆 Best Model: {best_model_name}")
        print(f"Best Test MAE: ${results[best_model_name]['test_mae']:.2f}")
        print(f"Best Test MAPE: {results[best_model_name]['test_mape']:.2f}%")

        return results, best_model_name

    def train_time_series_models(self, train_df):
        """Train traditional time series models for comparison"""
        print("\n" + "="*40)
        print("TRAINING TIME SERIES MODELS")
        print("="*40)

        # Prepare time series data
        ts_data = train_df.set_index('date')['sales']

        ts_results = {}

        # Simple Moving Average
        print("Training Simple Moving Average...")
        window_size = 30
        sma_forecast = ts_data.rolling(window=window_size).mean().iloc[-1]
        ts_results['Simple Moving Average'] = {
            'forecast': sma_forecast,
            'method': 'Simple Moving Average'
        }

        # Exponential Smoothing
        try:
            print("Training Exponential Smoothing...")
            exp_model = ExponentialSmoothing(ts_data, trend='add', seasonal='add', seasonal_periods=7)
            exp_fitted = exp_model.fit()
            ts_results['Exponential Smoothing'] = {
                'model': exp_fitted,
                'aic': exp_fitted.aic,
                'method': 'Exponential Smoothing'
            }
            print(f"Exponential Smoothing AIC: {exp_fitted.aic:.2f}")
        except Exception as e:
            print(f"Exponential Smoothing failed: {e}")

        # ARIMA Model (simplified)
        try:
            print("Training ARIMA model...")
            arima_model = ARIMA(ts_data, order=(1, 1, 1))
            arima_fitted = arima_model.fit()
            ts_results['ARIMA'] = {
                'model': arima_fitted,
                'aic': arima_fitted.aic,
                'method': 'ARIMA'
            }
            print(f"ARIMA AIC: {arima_fitted.aic:.2f}")
        except Exception as e:
            print(f"ARIMA training failed: {e}")

        return ts_results

    def evaluate_and_visualize(self, results, best_model_name, y_test, test_df):
        """Comprehensive evaluation and visualization"""
        print("\n" + "="*50)
        print("MODEL EVALUATION AND VISUALIZATION")
        print("="*50)

        best_predictions = results[best_model_name]['predictions']

        # Create comprehensive dashboard
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=['Actual vs Predicted', 'Residuals Over Time',
                          'Residuals Distribution', 'Model Comparison'],
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )

        # 1. Actual vs Predicted
        fig.add_trace(
            go.Scatter(x=test_df['date'], y=y_test, mode='lines', name='Actual Sales',
                      line=dict(color='blue')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=test_df['date'], y=best_predictions, mode='lines', name='Predicted Sales',
                      line=dict(color='red', dash='dash')),
            row=1, col=1
        )

        # 2. Residuals over time
        residuals = y_test - best_predictions
        fig.add_trace(
            go.Scatter(x=test_df['date'], y=residuals, mode='lines', name='Residuals',
                      line=dict(color='green')),
            row=1, col=2
        )
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=2)

        # 3. Residuals distribution
        fig.add_trace(
            go.Histogram(x=residuals, name='Residuals Distribution', nbinsx=30),
            row=2, col=1
        )

        # 4. Model comparison
        model_names = list(results.keys())
        mae_scores = [results[name]['test_mae'] for name in model_names]
        fig.add_trace(
            go.Bar(x=model_names, y=mae_scores, name='Test MAE',
                   marker_color=['gold' if name == best_model_name else 'lightblue'
                                for name in model_names]),
            row=2, col=2
        )

        fig.update_layout(height=800, showlegend=True, title_text="Sales Forecasting Model Evaluation")
        fig.show()

        # Feature importance (if available)
        if hasattr(self.best_model, 'feature_importances_'):
            self.plot_feature_importance()

        # Print detailed evaluation
        print(f"\nDetailed Evaluation for {best_model_name}:")
        print(f"Mean Absolute Error: ${results[best_model_name]['test_mae']:.2f}")
        print(f"Root Mean Square Error: ${results[best_model_name]['test_rmse']:.2f}")
        print(f"R² Score: {results[best_model_name]['test_r2']:.4f}")
        print(f"Mean Absolute Percentage Error: {results[best_model_name]['test_mape']:.2f}%")

        # Residual analysis
        print(f"\nResidual Analysis:")
        print(f"Mean residual: ${residuals.mean():.2f}")
        print(f"Residual standard deviation: ${residuals.std():.2f}")
        print(f"Residuals within 1 std: {np.sum(np.abs(residuals) <= residuals.std()) / len(residuals) * 100:.1f}%")

    def plot_feature_importance(self):
        """Plot feature importance for tree-based models"""
        if hasattr(self.best_model, 'feature_importances_'):
            importance_df = pd.DataFrame({
                'feature': self.feature_columns,
                'importance': self.best_model.feature_importances_
            }).sort_values('importance', ascending=True).tail(15)

            fig = go.Figure(go.Bar(
                x=importance_df['importance'],
                y=importance_df['feature'],
                orientation='h',
                marker_color='lightgreen'
            ))

            fig.update_layout(
                title='Top 15 Feature Importances',
                xaxis_title='Importance',
                yaxis_title='Features',
                height=500
            )

            fig.show()

            print("\nTop 10 Most Important Features:")
            for i, (_, row) in enumerate(importance_df.tail(10).iterrows(), 1):
                print(f"{i:2d}. {row['feature']}: {row['importance']:.4f}")

    def create_business_insights(self, df, results, best_model_name):
        """Generate business insights and recommendations"""
        print("\n" + "="*50)
        print("BUSINESS INSIGHTS AND RECOMMENDATIONS")
        print("="*50)

        # Calculate business metrics
        avg_daily_sales = df['sales'].mean()
        total_revenue = df['sales'].sum()
        revenue_growth = df['sales'].iloc[-30:].mean() / df['sales'].iloc[:30].mean() - 1

        # Seasonal insights
        best_day = df.groupby('day_of_week')['sales'].mean().idxmax()
        best_month = df.groupby('month')['sales'].mean().idxmax()
        holiday_lift = df[df['is_holiday_season']==1]['sales'].mean() / df[df['is_holiday_season']==0]['sales'].mean() - 1

        # Model accuracy insights
        forecast_accuracy = 100 - results[best_model_name]['test_mape']
        forecast_error = results[best_model_name]['test_mae']

        print("📊 BUSINESS PERFORMANCE SUMMARY:")
        print(f"• Total Revenue (3 years): ${total_revenue:,.2f}")
        print(f"• Average Daily Sales: ${avg_daily_sales:,.2f}")
        print(f"• Revenue Growth Rate: {revenue_growth:.1%}")
        print(f"• Holiday Season Lift: {holiday_lift:.1%}")

        print("\n📈 FORECASTING MODEL PERFORMANCE:")
        print(f"• Best Model: {best_model_name}")
        print(f"• Forecast Accuracy: {forecast_accuracy:.1f}%")
        print(f"• Average Prediction Error: ${forecast_error:.2f}")

        print("\n🎯 KEY BUSINESS INSIGHTS:")
        print(f"• Best performing day: {['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'][best_day]}")
        print(f"• Best performing month: {['', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'][best_month]}")
        print(f"• Weekend vs Weekday: ${df[df['is_weekend']==1]['sales'].mean():.2f} vs ${df[df['is_weekend']==0]['sales'].mean():.2f}")

        print("\n💡 STRATEGIC RECOMMENDATIONS:")
        print("1. Increase marketing spend during holiday seasons for maximum ROI")
        print("2. Optimize inventory for weekend demand patterns")
        print("3. Plan promotional campaigns around high-performing months")
        print("4. Use forecasting model for demand planning and budget allocation")

        return {
            'avg_daily_sales': avg_daily_sales,
            'total_revenue': total_revenue,
            'forecast_accuracy': forecast_accuracy,
            'best_model': best_model_name
        }

    def simulate_production_deployment(self):
        """Simulate production deployment scenarios"""
        print("\n" + "="*50)
        print("PRODUCTION DEPLOYMENT SIMULATION")
        print("="*50)

        # Simulate model serving performance
        sample_features = pd.DataFrame({
            col: np.random.randn(100) for col in self.feature_columns
        })

        # Measure prediction speed
        start_time = datetime.now()
        predictions = self.best_model.predict(sample_features)
        prediction_time = (datetime.now() - start_time).total_seconds()

        avg_prediction_time = prediction_time / len(sample_features) * 1000  # milliseconds

        print(f"⚡ PERFORMANCE METRICS:")
        print(f"• Batch prediction time: {prediction_time:.4f} seconds")
        print(f"• Average prediction time: {avg_prediction_time:.2f} ms per sample")
        print(f"• Throughput: {len(sample_features)/prediction_time:.0f} predictions/second")

        print(f"\n🚀 DEPLOYMENT READINESS:")
        print("✅ Model trained and validated")
        print("✅ Feature pipeline established")
        print("✅ Performance benchmarks met")
        print("✅ Business metrics calculated")

        print(f"\n📋 NEXT STEPS FOR PRODUCTION:")
        print("1. Set up automated data pipeline")
        print("2. Implement model monitoring dashboard")
        print("3. Create alerting for prediction drift")
        print("4. Schedule periodic model retraining")
        print("5. A/B test forecasting accuracy in production")

# Main execution function
def main():
    print("="*80)
    print("SALES FORECASTING WITH MLOPS PIPELINE")
    print("="*80)

    # Initialize forecasting system
    forecaster = SalesForecastingSystem()

# Main execution function
def main():
    print("="*80)
    print("SALES FORECASTING WITH MLOPS PIPELINE")
    print("="*80)

    # Initialize forecasting system
    forecaster = SalesForecastingSystem()

    # Step 1: Generate realistic sales data
    print("Step 1: Generating realistic sales data...")
    df = forecaster.generate_realistic_sales_data(periods=1095)  # 3 years of data

    # Step 2: Comprehensive EDA
    print("\nStep 2: Performing comprehensive EDA...")
    df_analyzed = forecaster.perform_comprehensive_eda(df)

    # Step 3: Advanced feature engineering
    print("\nStep 3: Advanced feature engineering...")
    df_features = forecaster.advanced_feature_engineering(df_analyzed)

    # Step 4: Prepare ML datasets
    print("\nStep 4: Preparing ML datasets...")
    X_train, X_test, y_train, y_test, train_df, test_df = forecaster.prepare_ml_datasets(df_features)

    # Step 5: Train multiple ML models
    print("\nStep 5: Training multiple ML models...")
    ml_results, best_ml_model = forecaster.train_multiple_models(X_train, y_train, X_test, y_test)

    # Step 6: Train time series models
    print("\nStep 6: Training traditional time series models...")
    ts_results = forecaster.train_time_series_models(train_df)

    # Step 7: Evaluate and visualize
    print("\nStep 7: Evaluating and visualizing results...")
    forecaster.evaluate_and_visualize(ml_results, best_ml_model, y_test, test_df)

    # Step 8: Generate business insights
    print("\nStep 8: Generating business insights...")
    business_insights = forecaster.create_business_insights(df_features, ml_results, best_ml_model)

    # Step 9: Simulate production deployment
    print("\nStep 9: Simulating production deployment...")
    forecaster.simulate_production_deployment()

    print("\n" + "="*80)
    print("SALES FORECASTING ANALYSIS COMPLETE!")
    print("="*80)
    print(f"Best Model: {best_ml_model}")
    print(f"Forecast Accuracy: {business_insights['forecast_accuracy']:.1f}%")
    print(f"Average Daily Sales: ${business_insights['avg_daily_sales']:,.2f}")
    print(f"Total Revenue (3 years): ${business_insights['total_revenue']:,.2f}")

    return forecaster, ml_results, business_insights

# Run the complete sales forecasting analysis
if __name__ == "__main__":
    forecaster, results, insights = main()
    print("Step 1: Generating realistic sales data...")
    df = forecaster.generate_realistic_sales_data(periods=1095)  # 3 years of data

    # Step 2: Comprehensive EDA
    print("\nStep 2: Performing comprehensive EDA...")
    df_analyzed = forecaster.perform_comprehensive_eda(df)

    # Step 3: Advanced feature engineering
    print("\nStep 3: Advanced feature engineering...")
    df_features = forecaster.advanced_feature_engineering(df_analyzed)

    # Step 4: Prepare ML datasets
    print("\nStep 4: Preparing ML datasets...")
    X_train, X_test, y_train, y_test, train_df, test_df = forecaster.prepare_ml_datasets(df_features)

    # Step 5: Train multiple ML models
    print("\nStep 5: Training multiple ML models...")
    ml_results, best_ml_model = forecaster.train_multiple_models(X_train, y_train, X_test, y_test)

    # Step 6: Train time series models
    print("\nStep 6: Training traditional time series models...")
    ts_results = forecaster.train_time_series_models(train_df)

    # Step

SALES FORECASTING WITH MLOPS PIPELINE
Step 1: Generating realistic sales data...
Generating 1095 days of realistic sales data...
Generated dataset shape: (1095, 22)
Date range: 2020-01-01 00:00:00 to 2022-12-30 00:00:00
Average daily sales: $1553.48

Step 2: Performing comprehensive EDA...

SALES FORECASTING - EXPLORATORY DATA ANALYSIS
Dataset shape: (1095, 22)
Date range: 2020-01-01 00:00:00 to 2022-12-30 00:00:00
Total sales: $1,701,056.11
Average daily sales: $1553.48
Sales standard deviation: $490.05



Key Insights:
1. Highest sales day of week: Wed ($1856.85)
2. Lowest sales day of week: Sun ($1268.11)
3. Best month: Jul ($1902.20)
4. Weekend vs weekday sales: $1280.80 vs $1662.13
5. Holiday season vs regular: $1610.28 vs $1542.15

Stationarity Test (Augmented Dickey-Fuller):
ADF Statistic: -2.2783
p-value: 0.1791
Series is non-stationary

Step 3: Advanced feature engineering...

Performing advanced feature engineering...
Feature engineering complete. Dataset shape: (1095, 48)

Step 4: Preparing ML datasets...

Preparing ML datasets with 30 day forecast horizon...
Cleaned dataset shape: (1006, 48)
Features used: 46
Training set: 804 samples
Test set: 202 samples
Test period: 2022-06-12 00:00:00 to 2022-12-30 00:00:00

Step 5: Training multiple ML models...

TRAINING MULTIPLE FORECASTING MODELS

Training Linear Regression...
Test MAE: $38.24
Test RMSE: $51.58
Test R²: 0.9740
Test MAPE: 1.90%
Training time: 0.03 seconds

Training Random Forest...
Test MAE: $71.96
Test RMSE: $106.68
T


Detailed Evaluation for Linear Regression:
Mean Absolute Error: $38.24
Root Mean Square Error: $51.58
R² Score: 0.9740
Mean Absolute Percentage Error: 1.90%

Residual Analysis:
Mean residual: $-4.17
Residual standard deviation: $51.54
Residuals within 1 std: 74.3%

Step 8: Generating business insights...

BUSINESS INSIGHTS AND RECOMMENDATIONS
📊 BUSINESS PERFORMANCE SUMMARY:
• Total Revenue (3 years): $1,701,056.11
• Average Daily Sales: $1,553.48
• Revenue Growth Rate: 234.7%
• Holiday Season Lift: 4.4%

📈 FORECASTING MODEL PERFORMANCE:
• Best Model: Linear Regression
• Forecast Accuracy: 98.1%
• Average Prediction Error: $38.24

🎯 KEY BUSINESS INSIGHTS:
• Best performing day: Wednesday
• Best performing month: July
• Weekend vs Weekday: $1280.80 vs $1662.13

💡 STRATEGIC RECOMMENDATIONS:
1. Increase marketing spend during holiday seasons for maximum ROI
2. Optimize inventory for weekend demand patterns
3. Plan promotional campaigns around high-performing months
4. Use forecasting mode


Key Insights:
1. Highest sales day of week: Wed ($1856.85)
2. Lowest sales day of week: Sun ($1268.11)
3. Best month: Jul ($1902.20)
4. Weekend vs weekday sales: $1280.80 vs $1662.13
5. Holiday season vs regular: $1610.28 vs $1542.15

Stationarity Test (Augmented Dickey-Fuller):
ADF Statistic: -2.2783
p-value: 0.1791
Series is non-stationary

Step 3: Advanced feature engineering...

Performing advanced feature engineering...
Feature engineering complete. Dataset shape: (1095, 48)

Step 4: Preparing ML datasets...

Preparing ML datasets with 30 day forecast horizon...
Cleaned dataset shape: (1006, 48)
Features used: 46
Training set: 804 samples
Test set: 202 samples
Test period: 2022-06-12 00:00:00 to 2022-12-30 00:00:00

Step 5: Training multiple ML models...

TRAINING MULTIPLE FORECASTING MODELS

Training Linear Regression...
Test MAE: $38.24
Test RMSE: $51.58
Test R²: 0.9740
Test MAPE: 1.90%
Training time: 0.10 seconds

Training Random Forest...
Test MAE: $71.96
Test RMSE: $106.68
T